# GWAS exploration

Polynote notebook for manual exploratory analysis of VEP annotations and vcf files



In [1]:
import org.apache.spark._
import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.ColumnName

In [2]:
import better.files._
import File._
import java.io.{File => JFile}

In [3]:
import org.apache.spark.sql.expressions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._

In [4]:
def display(f: File) = f.children.foreach(println)

In [5]:
val data = File("/data")
val gwas_path = data / "gwas" / "anton"
display(gwas_path)

/data/gwas/anton/vep
/data/gwas/anton/dante
/data/gwas/anton/variants
/data/gwas/anton/aligned
/data/gwas/anton/annotations
/data/gwas/anton/OLD
/data/gwas/anton/fastq


In [28]:
val dante = gwas_path / "dante"
display(dante)

/data/gwas/anton/dante/vcf
/data/gwas/anton/dante/result_alignment
/data/gwas/anton/dante/clean_data
/data/gwas/anton/dante/result_variation


Exploring variants
==================

In [20]:
val variants = gwas_path / "variants"
display(variants)

/data/gwas/anton/variants/smoove
/data/gwas/anton/variants/strelka_run


In [25]:
import org.apache.spark.SparkContext
import org.bdgenomics.adam.rdd.ADAMContext._
import org.bdgenomics.adam.rdd.read.AlignmentDataset


# DANTE

let's check DANTE variants first




## SNP variants



In [26]:
val dante_variants = dante / "result_variation"
display(dante_variants)

/data/gwas/anton/dante/result_variation/cnv
/data/gwas/anton/dante/result_variation/indel
/data/gwas/anton/dante/result_variation/sv
/data/gwas/anton/dante/result_variation/snp


In [54]:
val dante_snp_annotations = spark.readTSV( (dante_variants / "snp" / "750018002018_WGZ.snp.annot.csv").toString, sep=",", header = true)
dante_snp_annotations.show(10,1000)

+-----------------------+---------+--------------+-------------------------+------------+--------+------------+---------+---------+--------------+----+---------+----------------+-------+-------+----+-----+-----+---+---+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [53]:
//TODO: do some exploration

# Our Variants

Now let's check variants computed by our pipeline




In [17]:
val smoove = variants / "smoove" / "antonkulaga-smoove.vcf.gz"
val cnv = spark.sparkContext.loadVcf(smoove.toString)
cnv

RDDBoundVariantContextDataset with 194 reference sequences and 1 samples

In [27]:
cnv.dataset.show(10,1000)

+-------------+--------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Exploring annotations

exploring VEP annotations




In [38]:
val gene_names: DataFrame = spark.readTSV( "/data/sources/yspecies/data/input/genes/reference_genes.tsv", header = true)
 .withColumnRenamed("gene", "Gene")
println(gene_names.count)
gene_names.show(10,1000)

67996
+------------+---------------+-------+
|     species|           Gene| symbol|
+------------+---------------+-------+
|Homo_sapiens|ENSG00000242265|  PEG10|
|Homo_sapiens|ENSG00000139990|  DCAF5|
|Homo_sapiens|ENSG00000073921| PICALM|
|Homo_sapiens|ENSG00000139687|    RB1|
|Homo_sapiens|ENSG00000119977|  TCTN3|
|Homo_sapiens|ENSG00000145592|  RPL37|
|Homo_sapiens|ENSG00000242866|   STRC|
|Homo_sapiens|ENSG00000135506|    OS9|
|Homo_sapiens|ENSG00000150687| PRSS23|
|Homo_sapiens|ENSG00000162426|SLC45A1|
+------------+---------------+-------+
only showing top 10 rows



In [39]:
def with_names(df: DataFrame, how: String="inner") = df.join(gene_names, Seq("Gene"), how)

In [45]:
val anno_path = gwas_path / "vep"
display(anno_path)

/data/gwas/anton/vep/strelka


# SNPs



In [43]:
display(anno_path / "strelka" / "annotations")

/data/gwas/anton/vep/strelka/annotations/antonkulaga_variant_annotations.tsv
/data/gwas/anton/vep/strelka/annotations/antonkulaga_variant_annotations.tsv_summary.html


In [46]:
val columns = List("Uploaded_variation", "Location", "Allele", "Gene", "Feature", "Feature_type", "Consequence", "cDNA_position", "CDS_position", "Protein_position", "Amino_acids", "Codons", "Existing_variation", "IND", "ZYG", "IMPACT", "DISTANCE", "STRAND", "FLAGS", "VARIANT_CLASS", "SYMBOL", "SYMBOL_SOURCE", "HGNC_ID", "BIOTYPE", "CANONICAL", "MANE", "TSL", "APPRIS", "CCDS", "ENSP", "SWISSPROT", "TREMBL", "UNIPARC", "UNIPROT_ISOFORM", "GENE_PHENO", "SIFT", "PolyPhen", "EXON", "INTRON", "DOMAINS", "miRNA", "HGVSc", "HGVSp", "HGVS_OFFSET", "AF", "AFR_AF", "AMR_AF", "EAS_AF", "EUR_AF", "SAS_AF", "AA_AF", "EA_AF", "gnomAD_AF", "gnomAD_AFR_AF", "gnomAD_AMR_AF", "gnomAD_ASJ_AF", "gnomAD_EAS_AF", "gnomAD_FIN_AF", "gnomAD_NFE_AF", "gnomAD_OTH_AF", "gnomAD_SAS_AF", "MAX_AF", "MAX_AF_POPS", "CLIN_SIG", "SOMATIC", "PHENO", "PUBMED", "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE", "TRANSCRIPTION_FACTORS", "G2P_complete", "G2P_flag", "G2P_gene_req")

In [44]:
val snp_path = "/data/cromwell-executions/annotations/c2db854a-0256-4598-8497-bb5a0e69d528/call-vep_annotation/execution/antonkulaga_variant_annotations.tsv"
val annotations = spark.readTSV(snp_path).toDF(columns:_*)
annotations.show(10,10000)

+------------------+--------+------+---------------+---------------+-----------------+-------------------------+-------------+------------+----------------+-----------+------+------------------+-------+---+--------+--------+------+-----+-------------+-------+-------------+----------+----------------------------------+---------+----+---+------+----+----+---------+------+-------+---------------+----------+----+--------+----+------+-------+-----+-----+-----+-----------+---+------+------+------+------+------+-----+-----+---------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------+-----------+--------+-------+-----+------+----------+---------+------------+------------------+---------------------------------------------------------------------------------------------------------------------------------------------+------------+--------+------------+
|Uploaded_variation|Location|Allele|           Gene|        Feature|     Fea

In [47]:
annotations.select("G2P_flag").distinct

[G2P_flag: string]

## STRUCTURAL




In [7]:
val annotations = spark.readTSV( (anno_path / "test" / "antonkulaga_variant_annotations.tsv").toString, header = false).toDF(cols:_*)
annotations.show(10)

+------------------+--------+------+---------------+---------------+-----------------+--------------------+-------------+------------+----------------+-----------+------+------------------+--------+--------+------+-----+-------------+-------+-------------+----------+--------------------+---------+----+---+------+----+----+---------+------+-------+----------+----+--------+----+------+-------+-----+-----+-----+-----------+---+------+------+------+------+------+-----+-----+---------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------+-----------+--------+-------+-----+------+---------------+---------+------------+------------------+
|Uploaded_variation|Location|Allele|           Gene|        Feature|     Feature_type|         Consequence|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|Existing_variation|  IMPACT|DISTANCE|STRAND|FLAGS|VARIANT_CLASS| SYMBOL|SYMBOL_SOURCE|   HGNC_ID|             BIOTYPE|CANONIC

In [48]:
annotations.where($"gnomAD_AMR_AF" =!= "-").show(10,1000)

+------------------+--------+------+---------------+---------------+-----------------+-------------------------+-------------+------------+----------------+-----------+-------+----------------------+--------+--------+------+-----+-------------+----------+------------------------+----------+--------------------+---------+----+---+------+-----------+---------------+---------+----------+-------------+----------+---------------+---------+----+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+--------------------------+-----------------------------+-----------+------+------+------+------+------+------+------+------+---------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------+-----------+--------+-------+-----+------+----------+---------+------------+------------------+
|Uploaded_variation|Location|Al

In [9]:

val counts = annotations.select("Consequence", "Gene").groupBy("Consequence").agg(count($"Gene").as("count"))

counts.sort($"count".desc).show(1000,1000)

+----------------------------------------------------------------------+------+
|                                                           Consequence| count|
+----------------------------------------------------------------------+------+
|                                                        intron_variant|264592|
|                          intron_variant,non_coding_transcript_variant|178337|
|                                                 upstream_gene_variant| 76515|
|                                               downstream_gene_variant| 73663|
|                                 intron_variant,NMD_transcript_variant| 51102|
|                                                    intergenic_variant| 45047|
|                                             regulatory_region_variant| 37080|
|                                    non_coding_transcript_exon_variant|  7534|
|                                               TF_binding_site_variant|  7255|
|                                       

GET SNP ids

In [13]:
annotations

In [11]:
val frameshift = annotations.where($"Consequence".contains("frameshift"))
println(frameshift.count)
frameshift.show(1000,1000)

9
+----------------------------------+-------------------+----------------------+---------------+---------------+------------+-----------------------------------------+-------------+------------+----------------+-----------+------------------------+-----------------------+------+--------+------+------------+-------------------+------------+-------------+----------+-----------------------+---------+----+---+------+---------+---------------+---------+----------+-------------+----------+----+--------+----+------+-----------------------------------------+-----+---------------------------------+------------------------------------+-----------+------+------+------+------+------+------+------+------+---------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------+-----------+--------+-------+-----+------+----------+---------+------------+------------------+
|                Uploaded_variation|           Location|                A

In [37]:
frameshift

Clinical

In [12]:
val clinical = annotations.where($"CLIN_SIG" =!= "-")
println(clinical.count)
clinical.show(10,1000)

2912
+------------------+---------+------+---------------+---------------+-----------------+------------------------------------+-------------+------------+----------------+-----------+-------+-------------------+--------+--------+------+----------+-------------+----------+------------------------+---------+---------------+---------+-----------+---+------+-----------+---------------+---------+----------+-------------+----------+---------------+-------------+----+------+-----------------------------------------------------------------------------------------------------------------------------------+-----+-----------------------------+----------------------------+-----------+------+------+------+------+------+------+------+------+---------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------+-----------+--------+-------+-----+--------------------------+----------+---------+------------+------------------+
|Uploaded_variat